# Schema Export Notebook
Exportiere alle `CREATE TABLE`- und `CREATE VIEW`-Statements aus der MySQL-Datenbank und speichere sie in einer SQL-Datei.

In [3]:
import os
import pymysql

# Datenbank-Verbindungsdaten
db_host = os.getenv('DB_HOST', 'localhost')
db_name = os.getenv('DB_NAME', 'job_tracker')
db_user = os.getenv('DB_USER', 'root')
db_pass = os.getenv('DB_PASSWORD', '')

# Verbindung aufbauen
conn = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name)
cursor = conn.cursor()

# Tabellen und Views abfragen
cursor.execute("SHOW FULL TABLES WHERE Table_type = 'BASE TABLE' OR Table_type = 'VIEW';")
entries = cursor.fetchall()

# Statements sammeln
statements = []
for table, ttype in entries:
    obj = 'VIEW' if ttype.upper() == 'VIEW' else 'TABLE'
    # pma__ Tabellen überspringen
    if table.startswith('pma__'):
        continue
    cursor.execute(f"SHOW CREATE {obj} `{table}`;")
    result = cursor.fetchone()
    create_stmt = result[1]
    statements.append(create_stmt + ';')

# Ausgabedatei schreiben
output_path = 'schema_export.sql'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(statements))

print(f"Schema exportiert nach {output_path}")

# Verbindung schließen
cursor.close()
conn.close()

Schema exportiert nach schema_export.sql
